# 4_1_10_APOYOS_ADULTOS_MAYORES_PAREJA_DEP


In [ ]:
import traceback
from datetime import datetime, date
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "4_1_10_APOYOS_ADULTOS_MAYORES_PAREJA_DEP"

# Puedes ajustar esta fecha de referencia si quieres usar el año de la declaración
REFERENCE_DATE = date.today()

# ------------------------
# Funciones auxiliares
# ------------------------

def extraer_valor(doc, path):
    """
    Extrae un valor desde un path anidado (dict/list).
    """
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                # Tomamos el primer elemento si es lista
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except Exception:
        return None


def normalizar_texto(v):
    if v is None:
        return None
    return str(v).strip().upper()


def parse_fecha(fecha_raw):
    """
    Intenta parsear una fecha en formatos comunes (YYYY-MM-DD, DD/MM/YYYY, etc.).
    Retorna date o None.
    """
    if not fecha_raw:
        return None

    if isinstance(fecha_raw, dict):
        # Intentar campos típicos
        for key in ("fecha", "valor", "iso", "date"):
            if key in fecha_raw:
                fecha_raw = fecha_raw[key]
                break
        else:
            # Si no hay claves conocidas, tomar cualquier valor string
            for v in fecha_raw.values():
                fecha_raw = v
                break

    s = str(fecha_raw).strip()

    formatos = ["%Y-%m-%d", "%d/%m/%Y", "%d-%m-%Y", "%Y/%m/%d"]
    for fmt in formatos:
        try:
            return datetime.strptime(s, fmt).date()
        except ValueError:
            continue
    return None


def calcular_edad(fecha_nac, ref_date=REFERENCE_DATE):
    if not fecha_nac:
        return None
    try:
        anios = ref_date.year - fecha_nac.year
        if (ref_date.month, ref_date.day) < (fecha_nac.month, fecha_nac.day):
            anios -= 1
        return anios
    except Exception:
        return None


def obtener_personas_mayores_65(doc):
    """
    Regresa True si se identifica al menos una persona (pareja o dependiente)
    con 65 años o más. También indica si tuvimos al menos una fecha procesable.
    """
    edades = []

    # Pareja (un solo registro normalmente)
    pareja = extraer_valor(doc, "declaracion.situacionPatrimonial.datosPareja")
    if isinstance(pareja, dict):
        fn = parse_fecha(pareja.get("fechaNacimiento"))
        edades.append(calcular_edad(fn))
    elif isinstance(pareja, list):
        for p in pareja:
            if isinstance(p, dict):
                fn = parse_fecha(p.get("fechaNacimiento"))
                edades.append(calcular_edad(fn))

    # Dependientes económicos (lista)
    dependientes = extraer_valor(
        doc,
        "declaracion.situacionPatrimonial.datosDependientesEconomicos.dependienteEconomico"
    )
    if isinstance(dependientes, dict):
        fn = parse_fecha(dependientes.get("fechaNacimiento"))
        edades.append(calcular_edad(fn))
    elif isinstance(dependientes, list):
        for d in dependientes:
            if isinstance(d, dict):
                fn = parse_fecha(d.get("fechaNacimiento"))
                edades.append(calcular_edad(fn))

    edades_validas = [e for e in edades if e is not None]

    if not edades and (pareja or dependientes):
        # hay estructura pero ninguna fecha interpretable
        return False, False  # (hay_mayores65, hay_edades_validas)

    if not edades_validas:
        return False, False

    hay_mayores_65 = any(e >= 65 for e in edades_validas)
    return hay_mayores_65, True


def tiene_apoyo_adultos_mayores(doc):
    """
    Revisa en /apoyos/apoyo si existe un programa de Adultos Mayores
    cuyo beneficiario sea pareja o dependiente económico.
    """
    apoyos = extraer_valor(doc, "declaracion.intereses.apoyos.apoyo")

    if not apoyos:
        return False, False  # (hay_apoyo_adultos, hay_info_apoyos)

    if isinstance(apoyos, dict):
        apoyos = [apoyos]

    hay_info = False
    for a in apoyos:
        if not isinstance(a, dict):
            continue

        beneficiario = normalizar_texto(a.get("beneficiarioPrograma"))
        nombre_prog = normalizar_texto(a.get("nombrePrograma") or a.get("programa") or "")

        if beneficiario or nombre_prog:
            hay_info = True

        # buscamos algo tipo "ADULTO(S) MAYOR(ES)"
        if (beneficiario in ("PAREJA", "DEPENDIENTE", "DEPENDIENTE ECONOMICO",
                             "DEPENDIENTE ECONÓMICO")
            and "ADULTO" in nombre_prog and "MAYOR" in nombre_prog):
            return True, True

    return False, hay_info


# ------------------------
# Evaluación principal
# ------------------------

def evaluar_metrica(doc):
    hay_mayores_65, edades_validas = obtener_personas_mayores_65(doc)
    apoyo_adultos, info_apoyos = tiene_apoyo_adultos_mayores(doc)

    # 1. No hay personas de 65+ → NO_APLICA
    if not hay_mayores_65:
        # si ni siquiera tenemos edades válidas pero sí estructura, es SIN_DATO
        if not edades_validas and (extraer_valor(doc, "declaracion.situacionPatrimonial.datosPareja")
                                   or extraer_valor(doc, "declaracion.situacionPatrimonial.datosDependientesEconomicos")):
            return "SIN_DATO"
        return "NO_APLICA"

    # 2. Hay 65+ pero sin información de apoyos → SIN_DATO
    if not info_apoyos:
        return "SIN_DATO"

    # 3. Hay 65+ y sí se declaró apoyo de Adultos Mayores → CUMPLE
    if apoyo_adultos:
        return "CUMPLE"

    # 4. Hay 65+ y no se declaró apoyo de Adultos Mayores → NO_CUMPLE
    return "NO_CUMPLE"


# ------------------------
# Procesamiento MongoDB
# ------------------------

def procesar_metrica_4_1_10():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find(
            {},
            {
                "_id": 1,
                "declaracion.intereses.apoyos": 1,
                "declaracion.situacionPatrimonial.datosPareja": 1,
                "declaracion.situacionPatrimonial.datosDependientesEconomicos": 1,
            },
            no_cursor_timeout=True,
        )

        for doc in cursor:
            total += 1
            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error en doc {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True,
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}")
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except Exception:
        traceback.print_exc()
    finally:
        try:
            client.close()
        except:
            pass
        print("\nConexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_1_10()


Procesando métrica 4_1_10_APOYOS_ADULTOS_MAYORES_PAREJA_DEP...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


 > Procesados 2000 documentos...
 > Procesados 4000 documentos...
 > Procesados 6000 documentos...
 > Procesados 8000 documentos...
 > Procesados 10000 documentos...
 > Procesados 12000 documentos...
